In [1]:
import pandas as pd
import os

Introduction About the Data : The dataset The goal is to predict price of given diamond (Regression Analysis).

There are 10 independent variables (including id):

id : unique identifier of each diamond

carat : Carat (ct.) refers to the unique unit of weight measurement used exclusively to weigh gemstones and diamonds.

cut : Quality of Diamond Cut

color : Color of Diamond

clarity : Diamond clarity is a measure of the purity and rarity of the stone, graded by the visibility of these characteristics under 10-power magnification.

depth : The depth of diamond is its height (in millimeters) measured from the culet (bottom tip) to the table (flat, top surface)

table : A diamond's table is the facet which can be seen when the stone is viewed face up.

x : Diamond X dimension

y : Diamond Y dimension

x : Diamond Z dimension

Target variable:

price: Price of the given Diamond.

In [4]:
%pwd
os.chdir('../')

In [60]:
df = pd.read_csv('data/train.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193573 entries, 0 to 193572
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       193573 non-null  int64  
 1   carat    193573 non-null  float64
 2   cut      193573 non-null  object 
 3   color    193573 non-null  object 
 4   clarity  193573 non-null  object 
 5   depth    193573 non-null  float64
 6   table    193573 non-null  float64
 7   x        193573 non-null  float64
 8   y        193573 non-null  float64
 9   z        193573 non-null  float64
 10  price    193573 non-null  int64  
dtypes: float64(6), int64(2), object(3)
memory usage: 16.2+ MB


In [8]:
df[df.duplicated()]

,id,carat,cut,color,clarity,depth,table,x,y,z,price


In [9]:
df.isnull().sum()

id         0
carat      0
cut        0
color      0
clarity    0
depth      0
table      0
x          0
y          0
z          0
price      0
dtype: int64

In [11]:
df

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681


In [27]:
cut_map={"Fair":1,"Good":2,"Very Good":3,"Premium":4,"Ideal":5}
clarity_map = {"I1":1,"SI2":2 ,"SI1":3 ,"VS2":4 , "VS1":5 , "VVS2":6 , "VVS1":7 ,"IF":8}
color_map = {"D":1 ,"E":2 ,"F":3 , "G":4 ,"H":5 , "I":6, "J":7}

In [28]:
df['cut'] = df['cut'].map(cut_map)
df['clarity'] = df['clarity'].map(clarity_map)
df['color'] = df['color'].map(color_map)

In [61]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [62]:
X = df.drop(columns = ['id','price'])
y = df['price']

In [63]:
cat_feat = X.select_dtypes('object').columns
num_feat = X.select_dtypes(exclude='object').columns

In [64]:
cut_categories=["Fair", "Good","Very Good","Premium","Ideal"]
clarity_categories = ["I1", "SI2", "SI1", "VS2", "VS1", "VVS2", "VVS1", "IF"]
color_categories = ["D" ,"E" ,"F" , "G" ,"H", "I", "J"]

In [65]:
num_pipeline = Pipeline(
    steps =[
        ("imputer", SimpleImputer()),
        ("scalar", StandardScaler())
    ]
)
cat_pipeline = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("Ordinalencoder", OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories]))
    ]
)

In [66]:
preprocessor = ColumnTransformer(
    [
        ("num_pipeline",num_pipeline, num_feat),
        ("cat_pipeline",cat_pipeline, cat_feat)
    ]
)

In [67]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

In [70]:
X_train = preprocessor.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns = preprocessor.get_feature_names_out())

In [71]:
X_test = preprocessor.transform(X_test)
X_test = pd.DataFrame(X_test, columns = preprocessor.get_feature_names_out())

In [83]:
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [78]:
models= {
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet(),
    'RandomForestRegressor':RandomForestRegressor(),
    'XGBRegressor':XGBRegressor()
}

In [79]:
trained_model_list = []
model_list=[]
r2_list =[]

In [82]:
def evaluate_model(pred, test):
    mse = mean_squared_error(pred, test)
    mae = mean_absolute_error(pred, test)
    r2 = r2_score(pred, test)
    return mae, mse, r2

In [87]:
for model in list(models):
    regressor = models[model].fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    MAE, MSE, R2 = evaluate_model(y_test, y_pred)

    print("Model training performance for model: ", model)
    print("MSE: ",MSE),
    print("MAE: ", MAE),
    print("R2 SCORE: ", R2)

Model training performance for model:  LinearRegression
MSE:  1029473.3531156847
MAE:  675.0758270067478
R2 SCORE:  0.9362906819996047


Model training performance for model:  Lasso
MSE:  1029534.1175907081
MAE:  676.3788803086727
R2 SCORE:  0.9362869215688461
Model training performance for model:  Ridge
MSE:  1029482.8329815724
MAE:  675.1087261318776
R2 SCORE:  0.9362900953347935
Model training performance for model:  ElasticNet
MSE:  2291130.0570947696
MAE:  1051.172228186302
R2 SCORE:  0.8582126162411591
Model training performance for model:  RandomForestRegressor
MSE:  376680.2114500955
MAE:  310.84711351254975
R2 SCORE:  0.9766890135591169
Model training performance for model:  XGBRegressor
MSE:  345616.14680618217
MAE:  297.25052956080435
R2 SCORE:  0.9786114240487077
